# Running the `ialg` algorithm on the TSPLIB and the HardTSPLIB instances

Here, we want to compute the minimum number of SECs needed to prove optimality for some small famous instances in the TSPLIB [1] and some hard-to-solve instances in the HardTSPLIB [2]. We will use the `ialg` algorithm to compute the minimum number of SECs needed to prove optimality for these instances.

### References
[1] Reinelt, Gerhard. "TSPLIB—A traveling salesman problem library." ORSA journal on computing 3.4 (1991): 376-384.

[2] Vercesi, Eleonora, et al. "On the generation of metric TSP instances with a large integrality gap by branch-and-cut." Mathematical Programming Computation 15.2 (2023): 389-416.


In [8]:
# Suppress future warning of pandas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Things that you actually need
from ialg import ialg, mip
from cover import set_cover_subroutine
from utils import from_tsplib_file_to_graph
import pandas as pd
from tqdm import tqdm

## Download the data
You can download the files of the TSPLIB from [here](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/) and the HardTSPLIB from [here](https://github.com/eleonoravercesi/HardTSPLIB/). Then, you can put the files in the `data` folder.

## TSPLIB instances

These are the instances of the TSPLIB with a number of nodes that is $\leq 100$. Note that in this notebook you could either run the experiments for the small instances ( $n \leq 42$) or you can run all the experiments, but it will take more time.

In [9]:
tsplib_instances = [("burma14", 14), ("ulysses16", 16), ("gr17", 17), ("gr21", 21), ("ulysses22", 22),  ("gr24", 24),
                    ("fri26", 29), ("bayg29", 29), ("bays29", 29), ("dantzig42", 42), ("swiss42", 42)]
# In case you want to test also these instances
large_tsplib_instance = [("att48", 48), ("gr48", 48), ("hk48", 48), ("eil51", 51), ("berlin52", 52), ("brazil58", 58), ("st70", 70),
                     ("eil76", 76), ("pr76", 76), ("gr96", 96), ("rat99", 99), ("kroA100", 100), ("kroB100", 100),
                    ("kroC100", 100), ("kroD100", 100), ("kroE100", 100), ("rd100", 100)]
# Comment this if you want to run only the small instances
tsplib_instances += large_tsplib_instance

### With minimalization

In [10]:
minim = True

In [ ]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in tsplib_instances], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in tqdm(tsplib_instances):
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

  0%|          | 0/28 [00:00<?, ?it/s]

******* Instance: burma14 *******
TSP compute in 0.00585174560546875 seconds. TSP cost = 3323
With smart initialization, we begin with #SECs = 2
They are:
[0, 1, 7, 8, 9, 10]
[8, 9, 10]
Found a solution with #SECs: 2
Specifically, they are:
[0, 1, 7, 8, 9, 10]
[8, 9, 10]
Set parameter MIPGap to value 1e-16
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-13600, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xbbc18e87
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows

 11%|█         | 3/28 [00:00<00:01, 17.80it/s]

To cut off these partitions, require at least 5 SECs.
k* = 5 for S_family = [frozenset({0, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16}), frozenset({0, 1, 4, 5, 8, 9, 10, 11, 13, 16}), frozenset({0, 1, 3, 4, 5, 8, 9, 10, 11, 13, 16}), frozenset({0, 2, 3, 6, 7, 9, 10, 11, 12, 14, 15}), frozenset({12, 7, 6, 15})]
******* Instance: gr21 *******
TSP compute in 0.006322383880615234 seconds. TSP cost = 2707
With smart initialization, we begin with #SECs = 0
They are:
Found a solution with #SECs: 0
Specifically, they are:
Set parameter MIPGap to value 1e-16
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-13600, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matr

 21%|██▏       | 6/28 [00:00<00:01, 17.20it/s]

...

To cut off these partitions, require at least 8 SECs.
k* = 8 for S_family = [frozenset({0, 1, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47}), frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 46, 47}), frozenset({32, 2, 3, 4, 37, 6, 9, 11, 44, 46, 47, 21, 29}), frozenset({0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47}), frozenset({0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47}), frozenset({1, 34, 40, 41, 42, 10, 12, 14, 30}), frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 2

 54%|█████▎    | 15/28 [00:12<00:21,  1.62s/it]

To cut off these partitions, require at least 2 SECs.
k* = 2 for S_family = [frozenset({0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}), frozenset({19, 2, 35, 34})]
******* Instance: berlin52 *******
TSP compute in 0.07716965675354004 seconds. TSP cost = 7542
With smart initialization, we begin with #SECs = 1
They are:
[1, 6, 41]
Found smaller partition: 6 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 1 2
Found a solution with #SECs: 2
Specifically, they are:
[1, 6, 41]
[10, 11, 12, 13, 46, 50, 51, 25, 26, 27]
Set parameter MIPGap to value 1e-16
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-13600, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, u

 57%|█████▋    | 16/28 [00:13<00:16,  1.36s/it]

To cut off these partitions, require at least 2 SECs.
k* = 2 for S_family = [frozenset({0, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51}), frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49})]
******* Instance: brazil58 *******
TSP compute in 0.13296914100646973 seconds. TSP cost = 25395
With smart initialization, we begin with #SECs = 4
They are:
[6, 30, 37, 41]
[37, 6, 41, 30, 15]
[1, 40, 47, 53, 54]
[5, 13, 14, 16, 18, 20, 25, 27, 28, 32, 33, 35, 36, 44, 45, 55]
Found smaller partition: 14 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 4 2
Found smaller partition: 12 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 5 2
Found smaller partition: 11 -> 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 3 6 3
Foun

In [ ]:
df

### Without minimalization

In [ ]:
minim = False

In [ ]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in tsplib_instances], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in tqdm(tsplib_instances):
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

In [ ]:
df

## HardTSPLIB instances

HardTSPLIB is made of instances generated both at random and starting from instances of the TSPLIB.  We divide such instances into to, to make direct comparison with TSPLIB. `hardtsplib_instances_random` and `hardtsplib_instances_tsplib`. We will only run the algorithm on the instances that are feasible to run on our machine. 

In [ ]:
hardtsplib_instances_random = [("10001_hard", 10), ("10007_hard", 10), ("10008_hard", 10), ("10010_hard", 10), ("11675_hard", 11), ("12290_hard", 12), ("14850_hard", 14), ("15002_hard", 15), ("15005_hard", 15), ("15007_hard", 15), ("16038_hard", 16), ("20004_hard", 20), ("20007_hard", 20)]

hardtsplib_instances_random_large = [ ("20009_hard", 20), ("20181_hard", 20), ("25001_hard", 25), ("25004_hard", 25), ("25006_hard", 25), ("30001_hard", 30), ("30003_hard", 30), ("30005_hard", 30), ("33001_hard", 33), ("35002_hard", 35), ("35003_hard", 35), ("35009_hard", 35)]

# Comment this if you want to run only the small instances
hardtsplib_instances_random += hardtsplib_instances_random_large

hardtsplib_instances_tsplib = [("gr24_hard", 24), ("bayg29_hard", 29), ("bays29_hard", 29)]


## HardTSPLIB instances derived from TSPLIB

Note: here we run everuthing only with the minimalization procedure in place

In [ ]:
minim = True

### Comparison of the `ialg` algorithm on the TSPLIB and HardTSPLIB instances
In this case, we can see a line-by-line comparison between the values of TSPLIB and HardTSPLIB
    

In [ ]:
df = pd.DataFrame(index = [x[0].split("_")[0] for x in hardtsplib_instances_tsplib], columns=["S_min_TSPLIB", "S_min_HardTSPLIB", "b_prime_TSPLIB", "b_prime_HardTSPLIB", "runtime_TSPLIB", "runtime_HardTSPLIB", "BeB_n_nodes_TSPLIB", "BeB_n_nodes_HardTSPLIB"])
for instance_name, n in hardtsplib_instances_tsplib:
    instance_name_short = instance_name.split("_")[0]
    G_TSPLIB = from_tsplib_file_to_graph("./data/" + instance_name_short + ".tsp")
    (S_family_TSPLIB, size_S_family_TSPLIB, partitions_TSPLIB, b_TSPLIB, runtime_TSPLIB, num_nodes_TSPLIB) = ialg(G_TSPLIB, verbose=False, minimalize=minim)
    print("Instance:", instance_name_short)
    print("Time TSPLIB:", runtime_TSPLIB)
    G_HardTSPLIB = from_tsplib_file_to_graph("./data/" + instance_name + ".tsp")
    (S_family_HardTSPLIB, size_S_family_HardTSPLIB, partitions_HardTSPLIB, b_HardTSPLIB, runtime_HardTSPLIB, num_nodes_HardTSPLIB) = ialg(G_HardTSPLIB, verbose=False, minimalize=minim)
    print("Time HardTSPLIB:", runtime_HardTSPLIB)
    df.loc[instance_name_short] = [size_S_family_TSPLIB, size_S_family_HardTSPLIB, b_TSPLIB, b_HardTSPLIB, runtime_TSPLIB, runtime_HardTSPLIB, num_nodes_TSPLIB, num_nodes_HardTSPLIB]

In [ ]:
df


## Hard instances derived from random instances

In [ ]:
minim = True

In [ ]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in hardtsplib_instances_random], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in tqdm(hardtsplib_instances_random):
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim, smart_initialize=True)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

In [ ]:
df